In [54]:
import numpy as np
import igl
import meshplot as mp
import scipy.sparse as sp
from sksparse.cholmod import cholesky
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from numba import jit

In [55]:
v, f = igl.read_triangle_mesh('data/hand.off')

# label = 0 is free points, label >=1 is handle points
labels = np.load('data/hand.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()
vnum = v.shape[0]
free_vids = np.where(labels == 0)[0]
handle_vids = np.where(labels >= 1)[0]
# print(labels.shape)
# print(free_vids.shape)
# print(handle_vids.shape)
# print(np.linalg.norm(igl.per_vertex_normals(v,f), axis = 1))

In [56]:
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    # vertex ids of this segment
    slices = (labels==s)
    # rotation operator
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    # translation
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    # rotation (act on v's copy)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    # new coordinates of all vertices
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()

    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [57]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

## Step 1: Removal of high-frequency details
## Step 2: Deforming the smooth mesh

In [58]:
# cot matrix
Cot = igl.cotmatrix(v, f)
# mass matrix
M = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
# inverse of M
MINV = sp.diags(1 / M.diagonal())
A = Cot.T@MINV@Cot
# partitioned matrix
Aff = A[free_vids, :][:, free_vids]
Afc = A[free_vids, :][:, handle_vids]

In [59]:
def smooth_deform(_vertices):
    xc = _vertices[handle_vids]
    xf = sp.linalg.spsolve(Aff, -Afc@xc)
    x = np.zeros((vnum,3))
    x[free_vids] = xf
    x[handle_vids] = xc
    return x
    

## Step 3: Transferring high-frequency details to the deformed surface

In [60]:
# get displacement between S1 - B1 on local frame of each vertex
def get_local_dis(_s1_vertices, _b1_vertices):
    # displacements s1-b1
    dis1s = _s1_vertices - _b1_vertices
    # to build local frame on each vertex (normal and tangent surface) of b1
    # normals are already normalized
    b1_normals = igl.per_vertex_normals(_b1_vertices, f)
    # Pythagorean theorem to find the adjacent edge with longest projection on the tangent surface (on b1)
    nei_vids_by_vertex = igl.adjacency_list(f)
    nei_vid_select_by_vertex = np.zeros(vnum, dtype = int)
    dxs = np.zeros(vnum)
    dys = np.zeros(vnum)
    dns = np.zeros(vnum)
    for vid in range(vnum):
        nei_vids = nei_vids_by_vertex[vid]
        nei_coords = _b1_vertices[nei_vids]
        nei_edges = nei_coords - _b1_vertices[vid]
        nei_edges_len = np.linalg.norm(nei_edges, axis = 1)
        normal = b1_normals[vid]
        # lengths of edges projected to normal direction (inner product)
        nei_edges_normal_proj = nei_edges @ normal
        nei_edges_tangent_proj_sq = np.power(nei_edges_len, 2) - np.power(nei_edges_normal_proj, 2)
        # find the neighbor vid with maximum tangent projection length
        neiid_select = np.argmax(nei_edges_tangent_proj_sq)
        nei_vid_select = nei_vids[neiid_select]
        nei_vid_select_by_vertex[vid] = nei_vid_select
        nei_edge_select = nei_edges[neiid_select]
        y_base = np.cross(normal, nei_edge_select)
        # normalize
        y_base = y_base / np.linalg.norm(y_base)
        x_base = np.cross(y_base, normal)
        # coordinates of displacement in this local frame
        dis1 = dis1s[vid]
        dxs[vid] = np.dot(dis1,x_base)
        dys[vid] = np.dot(dis1, y_base)
        dns[vid] = np.dot(dis1, normal)
    return dxs, dys, dns, nei_vid_select_by_vertex

In [61]:
smooth1_vertices = smooth_deform(v)
dxs, dys, dns, nei_vid_select_by_vertex = get_local_dis(v, smooth1_vertices)
# print(smooth1_vertices)

In [62]:
# b2 is new smooth mesh
def transfer_details(_b2_vertices):
    # normals are already normalized
    b2_normals = igl.per_vertex_normals(_b2_vertices, f)
    dis2s = np.zeros((vnum, 3))
    for vid in range(vnum):
        nei_vid_select = nei_vid_select_by_vertex[vid]
        nei_edge_select = _b2_vertices[nei_vid_select] - _b2_vertices[vid]
        # build b2 local frame at this vertex
        normal = b2_normals[vid]
        y_base = np.cross(normal, nei_edge_select)
        y_base = y_base / np.linalg.norm(y_base)
        x_base = np.cross(y_base, normal)
        dis2s[vid, :] = dxs[vid]*x_base + dys[vid]*y_base + dns[vid]*normal
    return _b2_vertices + dis2s

## Performance

In [63]:
# # In this section, modify the smooth_deform and transfer_details function 
# ## by Cholesky

factor = cholesky(Aff)
# accelerated version
def smooth_deform(_vertices):
    xc = _vertices[handle_vids]
    xf = factor(-1*Afc@xc)
    x = np.zeros((vnum,3))
    x[free_vids] = xf
    x[handle_vids] = xc
    return x

# accelerated version

def transfer_details(_b2_vertices):
    # normals are already normalized
    b2_normals = igl.per_vertex_normals(_b2_vertices, f)
    # dis2s = np.zeros((vnum, 3))
    nei_edges_select = _b2_vertices[nei_vid_select_by_vertex] - _b2_vertices
    y_bases_raw = np.cross(b2_normals, nei_edges_select)
    y_norms = np.linalg.norm(y_bases_raw, axis = 1)
    y_norms_recip = np.reciprocal(y_norms)
    # normalize
    y_bases = np.einsum('i,ij->ij', y_norms_recip, y_bases_raw)
    x_bases = np.cross(y_bases, b2_normals)
    dis2s = np.einsum('i,ij->ij', dxs, x_bases) + np.einsum('i,ij->ij', dys, y_bases) + np.einsum('i,ij->ij', dns, b2_normals)                          
    return _b2_vertices + dis2s

C:\Users\13400\AppData\Local\Temp/ipykernel_11328/983807932.py:4: CholmodTypeConversionWarning: converting matrix of class csr_matrix to CSC format
  factor = cholesky(Aff)


In [68]:

def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    # t0 = time.time()
    smooth2_vertices = smooth_deform(target_pos)
    # t1 = time.time()
    # print("smooth time", t1-t0)
    res = transfer_details(smooth2_vertices)
    # t2 = time.time()
    # print("tranfer time", t2-t1)
    return smooth2_vertices
# (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer


In [69]:
## Widget UI

p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f(s, x, y, z, α, β, γ)>